In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import os
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import math
import json
import numpy as np
import pickle

In [112]:
with open('/home/martin/Coding/DAS-political-confesions/FlaskApp/openaiKey.txt', 'r') as file:
    key_content = file.read()

OPENAI_API_KEY = str(key_content.strip())

In [113]:
with open('contexts_embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
    
with open('contexts.pkl', 'rb') as f:
    contexts = pickle.load(f)

In [114]:
json_file_path = "/home/martin/Coding/DAS-political-confesions/embedding/stopwords-cs.json"
# Open and load the JSON file
with open(json_file_path, "r", encoding="utf-8") as file:
    stopwords = json.load(file)
    
special_characters_mapping = {'š': 's', 'č': 'c', 'ě': 'e', 'ž': 'z', 'ř': 'r', 'ý': 'y', 'á': 'a', 'í': 'i', 'é': 'e', 'ň': 'n'}
special_characters_mapping.update({char.upper(): replacement.upper() for char, replacement in special_characters_mapping.items()})

def preprocess_text(text, stopwords):
    
#     for special_char, replacement in special_characters_mapping.items():
#         text = text.replace(special_char, replacement)
    
    words = text.split()

#     words = [word[0:4] for word in words if len(word) > 7]
    
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

In [115]:
model_name = "ufal/robeczech-base" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

feature_extraction_pipeline = pipeline('feature-extraction', model=model_name, tokenizer=tokenizer)  # 'device=0' for GPU, remove for CPU

Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [116]:
print(embeddings['slovakia_elections'])

[[-4.03628025e-02  5.36570216e-02  1.04333167e-03 -5.89159659e-02
   1.92006809e-02 -5.72244334e-02 -6.43231083e-02 -1.71656231e-02
   4.15612581e-02 -1.03746231e-02  6.90651411e-02 -9.38402009e-02
   3.00718857e-02  5.66149576e-02  2.68574475e-02 -1.88042546e-01
  -8.07171307e-03  2.77793469e-02  4.54649230e-02 -1.48850735e-01
   5.82393823e-02  6.45200362e-03  2.26554242e-02  6.45743286e-02
   8.90842038e-03 -2.40720595e-03 -5.39948161e-03  1.87258760e-02
   3.17098483e-02 -1.87978854e-02  2.56176807e-02 -5.87608113e-02
   4.78304324e-02  4.42866770e-03  9.79728583e-02 -9.02278918e-02
  -6.06011378e-03  1.07611751e-02 -6.94195352e-02  3.73238577e-02
  -2.65616262e-01  8.12871451e-02 -1.31005907e-01 -8.25457039e-02
  -1.64747529e-02  1.94960162e-03 -1.57446020e-02  7.77087138e-03
   6.66404108e-02  3.90221721e-01  1.10211283e-01  1.41016036e-02
  -1.04439083e-01 -1.13709966e-01  3.23126016e-01  1.49577259e-01
   5.15624249e-02 -8.80149047e-02 -8.51494326e-02 -3.92835758e-02
  -1.01502

In [125]:
prompt = "Co si myslíš o Slovenských volbách a Robertu Ficovi? Jaké vztahy máš s Ficem?"

In [141]:
#     text = preprocess_text(text, stopwords)
def get_embedding(prompt):
    chunk_size = 200  # Set your desired chunk size

    # Split the text into non-overlapping chunks
    chunks = [prompt[i:i + chunk_size] for i in range(0, len(prompt), chunk_size)]

        # Initialize an empty list to store chunk embeddings
    chunk_embeddings = []

        # Process each chunk and obtain embeddings
    for chunk in chunks:
        chunk_embedding = feature_extraction_pipeline(chunk)
        chunk_embedding = np.mean(chunk_embedding[0], axis=0)
        chunk_embeddings.append(chunk_embedding)

        # Aggregate embeddings (e.g., by averaging)
    prompt_embedding = np.mean(chunk_embeddings, axis=0)
    prompt_embedding = np.array(prompt_embedding).reshape(1, -1)
    return prompt_embedding

In [146]:
def get_class(embeddings, prompt_embedding):
    prompt_class = None
    max_sim = -1
    for i, (emb_name, emb_t) in enumerate(embeddings.items()):
        sim = cosine_similarity(prompt_embedding, emb_t)
        if sim > max_sim:
            prompt_class = emb_name
            max_sim = sim
    return prompt_class

In [128]:
print(prompt_class)

slovakia_elections


In [129]:
print(contexts[prompt_class])

Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku „dobrou vládu“ v naději, že Česká a Slovenská republika udrží své skvělé vztahy, což není zcela jisté.


In [130]:
llm = ChatOpenAI(temperature=0.75, model_name='ft:gpt-3.5-turbo-1106:personal::8RjGTQTI', openai_api_key = OPENAI_API_KEY)

In [131]:
formatted_template = f'Odpověz jako premiér České reubliky Petr Fiala: {prompt}. Kontext konverzace: {contexts[prompt_class]}'

In [107]:

resp = conversation.run([HumanMessage(content=formatted_template)])
print(resp)

Robert Fico je novým slovenským premiérem. Po volební noci mu pogratuloval a popřál Slovensku dobrou vládu s nadějí, že Česká a Slovenská republika udrží své skvělé vztahy.


In [139]:
def change_template(template, context):
    return """You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:
{history}
Human: {input}
Context: """ + context + """
Petr Fiala:"""

In [151]:
from langchain.prompts.prompt import PromptTemplate

template = """You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:
{history}
Human: {input} 
Petr Fiala:"""
print(template)
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="Petr Fiala"),
)


You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:
{history}
Human: {input} 
Petr Fiala:


In [150]:
prompt = "Ani v kontextu války na Ukrajině s ním nemáš problém?"

prompt_emb = get_embedding(prompt)
prompt_class = get_class(embeddings, q_emb)

formatted_prompt = f"{prompt}. Context: {contexts[prompt_class]}"


In [137]:
conversation.predict(input=prompt)



> Entering new ConversationChain chain...
Prompt after formatting:
You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:

Human: Co si myslíš o Slovenských volbách a Robertu Ficovi? Jaké vztahy máš s Ficem?
Context: Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku „dobrou

'Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy.'

In [138]:
conversation.predict(input="Nevadí ti, náklonost Roberta Fica k Rusku?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:
Human: Co si myslíš o Slovenských volbách a Robertu Ficovi? Jaké vztahy máš s Ficem?
Petr Fiala: Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy.
Human: Nevadí ti, náklonost Roberta Fica k Rusku?
Context: Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za F

'Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy. Nemám důvod Robertovi Ficovi nevěřit, myslím, že Slovensko s ním bude v dobrých rukách a že zůstanou zachovány dobré vztahy.'

In [147]:
prompt2 = "Ani v kontextu války na Ukrajině s ním nemáš problém?"
q_emb = get_embedding(prompt2)
q_class = get_class(embeddings, q_emb)


ukraine
Po mnoha letech provokací z ruské strany zahájilo Rusko 24. února 2022 rozsáhlou invazi na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se dokázala úspěšně bránit a zatlačit ruskou armádu a osvobozuje stále větší část svého území. Mnoho evropských zemí a USA odsoudili ruský útok a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po celém internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Mnoho politiků volá po mírových rozhovorech, které by znamenaly odevzdání některých ukrajinských území Rusku, což je pro Ukrajince nepřijatelné. Na druhou stranu ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinský prezident Zelenskyj je od začátku invaze symbolem svobody. Na druhé straně byl ruský prezident Putin symbolem autokracie, propagandy a extremistů. Válk

In [148]:
conversation.predict(input="Nevadí ti, náklonost Roberta Fica k Rusku?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are czech prime minister Petr Fiala impersonator. Answer questions in czech as if you were him, given a context of the conversation. 

Current conversation:
Human: Co si myslíš o Slovenských volbách a Robertu Ficovi? Jaké vztahy máš s Ficem?
Petr Fiala: Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy.
Human: Nevadí ti, náklonost Roberta Fica k Rusku?
Petr Fiala: Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy. Nemám důvod Robertovi Ficovi nevěřit, myslím, že Slovensko s ním bude v dobrých rukách a že zůstanou zachovány dobré vztahy.
Human: Nevadí ti, náklonost Roberta Fica k Rusku?
Context: Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLA

'Po volbách jsem Robertu Ficovi blahopřál, doufám, že Česko a Slovensko udrží dobré vztahy. Nemám důvod Robertovi Ficovi nevěřit, myslím, že Slovensko s ním bude v dobrých rukách a že zůstanou zachovány dobré vztahy.'